### Set Up

#### Standard library imports

In [ ]:
import os
import random
import datetime
import time
import sys

#### Third party imports 

In [ ]:
import pydot
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_probability as tfp
import plotly.express as px

#### Local imports

In [ ]:
import modules.batch as batch
import modules.midi_related as midi
import modules.preprocessing as prep
import modules.subclasses as sub

#### Extensions and autoreload

In [ ]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2

#### Dis-/enable GPU

In [ ]:
disable_gpu = False
debugging = False

if disable_gpu:
    # Hide GPU from visible devices
    tf.config.set_visible_devices([], 'GPU')
    if debugging:
        # To find out which devices your operations and tensors are assigned to
        tf.debugging.set_log_device_placement(True)

        # Create some tensors and perform an operation
        a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
        b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
        c = tf.matmul(a, b)

        print(c)
        tf.debugging.set_log_device_placement(False)

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')

#### Setting relative directories

In [ ]:
Working_Directory = os.getcwd()
Project_Directory = os.path.abspath(os.path.join(Working_Directory,'..'))
Music_In_Directory = Project_Directory + "/data/chopin_midi/"
Output_Directory = Project_Directory + "/outputs/"
Model_Directory = Output_Directory + "models/"
Checkpoint_Directory = Model_Directory + "ckpt/"
Numpy_Directory = Model_Directory + "arrays/"
Music_Out_Directory = Output_Directory + "midi/"
Music_Out_Training_Directory =  Music_Out_Directory + "train/"
Music_Out_Genereating_Directory = Music_Out_Directory + "generated/"
#Log_Directory = Output_Directory + "logs/fit/"

#### Tensorboard callback

In [ ]:
# Currently not in use

#current_time_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#log_dir = Log_Directory + current_time_str
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

### Data preperation

#### Load pieces (i.e. import midi files)

In [ ]:
# First checkt that importing single midi (i.e. Chopin Op 28 No.4) works
chop2804 = midi.midiToNoteStateMatrix(Music_In_Directory + "chop2804.mid", verbose=False, verbose_ts=False) 
# verbose = true, should help understand how the import works
# verbose_ts = true, prints resolution of midi file

In [ ]:
# Import all Chopin data
min_time_steps = 128 # only files with at least this many 48th note steps are saved

# Gather the training pieces from the specified directory
training_pieces={}
training_pieces = {**training_pieces, **midi.loadPieces(Music_In_Directory, min_time_steps, False)}
print('Number of total pieces = ', len(training_pieces))    

In [ ]:
# Check if writing back as midi file works
check_backtransform_midi = False #True
if check_backtransform_midi:
    out = tf.convert_to_tensor(training_pieces['chop2804'], dtype=tf.int32)
    midi.noteStateMatrixToMidi(out, Music_Out_Directory + "chop2804_backtransformed", tickscale = 33)

In [ ]:
# Create table of piece names for midi files loaded
df_chopin_opus = pd.read_csv(Project_Directory + '/data/chopin_title_opus.csv')
op_numbers = [int(str(k)[4:6]) for k in training_pieces.keys()]
op_numbers_unique = set(op_numbers)
df_chopin_opus_midi = df_chopin_opus.loc[df_chopin_opus['Op'].isin(op_numbers_unique)]

In [ ]:
# Define genres used for training
genres = ['Ballade', 'Etudes', 'promptu', 'Mazurkas', 
          'Nocturnes', 'Preludes',  'Sonata', 'Waltzes']
df_pieces_summary = pd.concat([midi.get_piece_summary_df(i, df_chopin_opus_midi, op_numbers) for i in genres])
df_genre_summary = df_pieces_summary[['MIDI files', 'genre', 'Op']] \
                    .groupby(['genre']) \
                    .agg({'Op': list, 'MIDI files': sum})
df_genre_summary.rename(index={'Ballade':'Ballades','promptu':'Impromptus', 'Sonata' : 'Sonatas'}, inplace=True)
df_genre_summary = df_genre_summary.sort_index()
#genre_summary.style.set_properties(subset=['Op'], **{'width-min': '300px'})

In [ ]:
# Define pieces for each genre
ballades = midi.get_subset_training_pieces_for_genre('Ballades', training_pieces, df_genre_summary)
etudes = midi.get_subset_training_pieces_for_genre('Etudes', training_pieces, df_genre_summary)
impromptus = midi.get_subset_training_pieces_for_genre('Impromptus', training_pieces, df_genre_summary)
mazurkas = midi.get_subset_training_pieces_for_genre('Mazurkas', training_pieces, df_genre_summary)
nocturnes = midi.get_subset_training_pieces_for_genre('Nocturnes', training_pieces, df_genre_summary)
preludes = midi.get_subset_training_pieces_for_genre('Preludes', training_pieces, df_genre_summary)
sonatas = midi.get_subset_training_pieces_for_genre('Sonatas', training_pieces, df_genre_summary)
waltzes = midi.get_subset_training_pieces_for_genre('Waltzes', training_pieces, df_genre_summary)

In [ ]:
# Update training_pieces to only use pieces from defined genres 
op_train = [item for sublist in df_genre_summary['Op'] for item in sublist]
training_keys = [k for k in training_pieces.keys() if int(str(k)[4:6]) in op_train]
training_pieces = {k:v for k, v in training_pieces.items() if k in training_keys}
#training_pieces = mazurkas
print('Number of total pieces left = ', len(training_pieces))    

#### Train/Validation pieces split

In [ ]:
# Either select one validation and one training piece or set aside a random set of pieces for validation purposes
single_piece = False
if single_piece:
    validation_pieces = {'chop2803' : training_pieces['chop2803']}
    training_pieces   = {'chop2804' : training_pieces['chop2804']}
else:
    num_validation_pieces = len(training_pieces) // 10

    validation_pieces={}
    for v in range(num_validation_pieces):
        index = random.choice(list(training_pieces.keys()))
        validation_pieces[index] = training_pieces.pop(index)

In [ ]:
print('Number of training   pieces = ', len(training_pieces))    
print('Number of validation pieces = ', len(validation_pieces))     

#### Check that features (X) and lables (y) generation work

In [ ]:
# Generate sample Note State Matrix for dimension measurement and numerical checking purposes
sample_size = 16
sample_num_timesteps = 144
Midi_low = 21
Midi_high = 108

y = batch.getPieceBatch(training_pieces, sample_size, sample_num_timesteps) 
X = prep.inputKernel(y, Midi_low, Midi_high)


print('Dimensions y: (sample_size, num_notes, num_timesteps, play_articulate_velocity) = ', y.shape)
print('Dimensions X: (sample_size, num_notes, num_timesteps, feature_dim             ) = ', X.shape)

In [ ]:
# Feature vector for the 1st batch the 53rd note and the 51st 48th-note timestep (cast to int)
prep.noteRNNInputSummary(X[0,30,50,:])

In [ ]:
# Compare to all notes played at timestep 51
[(i,j) for i,j in enumerate(list(tf.cast(y[0,:,50,:], dtype=tf.int32).numpy())) if j[0]== 1]

#### Test generating midi files from y_train 

In [ ]:
# Backtransform the 128 48th notes from the first random sampled training_piece to MIDI
test_midi = tf.cast(tf.transpose(y, perm=[0,2,1,3])[0,:,:,:], dtype=tf.int32).numpy()
midi.noteStateMatrixToMidi(test_midi, Music_Out_Directory + "random_test")

#### Check number of notes played, articulated and velocity per timestep

In [ ]:
test = tf.transpose(y, perm=[0,2,1,3])[0,:,:,:].numpy()
notes_per_timestep = [(i,tf.reduce_sum(test[i,:,0]).numpy()) for i in range(test.shape[0])]
articulate_per_timestep = [(i,tf.reduce_sum(test[i,:,1]).numpy()) for i in range(test.shape[0])]
mean_tempo_per_timestep = [(i,tf.reduce_mean(test[i,np.array(test[i,:,0], dtype=bool),2]).numpy()) for i in range(test.shape[0])]

### Model architecture

#### Parameters

In [ ]:
num_notes = 88      # X.shape[1] = Midi_high + 1 - Midi_low 
num_timesteps = -1  # keep the num_timesteps variable, in training set to X.shape[2]
input_size = 108     # X.shape[3]
drop_out_rate = 0.5

num_t_units = [128, 128] # [256, 128]
num_n_units = [64,  64] # [128,  64]
dense_units = 3    #(play,articulate,velocity)

TIME_BATCH_SHAPE = (num_timesteps, input_size)
NOTE_BATCH_SHAPE = (num_notes, num_t_units[1])

#### Inputs

In [ ]:
inputs = tf.keras.Input(shape=(num_notes, None, input_size), name="inputs")
inputs_shape = sub.GetShape()(inputs)

#### Timewise LSTM

In [ ]:
# Reshaping
x = sub.BatchReshape(TIME_BATCH_SHAPE, False)(inputs)

# Timewise LSTMs
x = tf.keras.layers.LSTM(num_t_units[0], return_sequences=True, dropout= drop_out_rate)(x)
x = tf.keras.layers.LSTM(num_t_units[1], return_sequences=True, dropout= drop_out_rate)(x)

#### Notewise LSTM

In [ ]:
# Reshaping
x = sub.BatchReshape([num_notes, num_timesteps , num_t_units[1]], True)(x)
x = tf.keras.layers.Permute((2,1,3))(x)
x = sub.BatchReshape(NOTE_BATCH_SHAPE, False)(x)

# Notewise LSTMs
x = tf.keras.layers.LSTM(num_n_units[0], return_sequences=True, dropout= drop_out_rate)(x)
x = tf.keras.layers.LSTM(num_n_units[1], return_sequences=True, dropout= drop_out_rate)(x)

#### Simulate conditional probabilty using dense layers

In [ ]:
x = sub.SliceNotesTensor()(x)

x_tmp = tf.keras.layers.Dense(units=dense_units, activation=None)(x[0])
note = sub.SampleNote()(x_tmp)

x_list = [x_tmp]
note_list = [sub.SliceNoteVelocityTensor()(note)]

for n in range(1,len(x)):
    x_tmp = tf.keras.layers.Concatenate(axis=-1)([x[n], note])
    x_tmp = tf.keras.layers.Dense(units=dense_units, activation=None)(x_tmp)
    note = sub.SampleNote()(x_tmp)
    x_list.append(x_tmp)
    note_list.append(sub.SliceNoteVelocityTensor()(note))

#### Outputs

In [ ]:
# Output 1    
x_list = sub.ExpandDims()(x_list, axis=1)
x = tf.keras.layers.Concatenate(axis=1)(x_list)
x = sub.BatchReshape([num_timesteps, num_notes , dense_units], True)(x, inputs_shape)
x_1, x_2 = sub.SliceNotesVelocityTensor()(x)
output_1 = tf.keras.layers.Permute((2,1,3), name='play_articulate_prob')(x_1)

#Output 2
x_2 = sub.BackTransformVelocity()(x_2)
output_2 = tf.keras.layers.Permute((2,1,3), name='velocity')(x_2)

# Output 3
note_list = sub.ExpandDims()(note_list, axis=1)
note = tf.keras.layers.Concatenate(axis=1)(note_list)
note = sub.BatchReshape([num_timesteps, num_notes , 2], True)(note, inputs_shape)
output_3 = tf.keras.layers.Permute((2,1,3), name='play_articulate_sampled')(note)

#### Full LSTM

In [ ]:
lstm = tf.keras.Model(inputs, [output_1, output_2, output_3], name="full")
lstm.summary()

In [ ]:
plot = False
if plot:
    plot_res = tf.keras.utils.plot_model(lstm)

### Model training

#### Train model

In [ ]:
num_timesteps = 144

# loss
losses = (sub.CustomSigmoidFocalCrossEntropy(from_logits = True, 
                                           gamma = 0, 
                                           alpha = 0),
          sub.MeanSquaredErrorVelocity())

# metric
metrics = (sub.CustomBinaryAccuracy(threshold=0.5), 
           tfa.metrics.MultiLabelConfusionMatrix(num_classes=2),
           sub.root_mean_squared_error_velocity_metric)

# optimizer
optimizer = tf.keras.optimizers.Adadelta(learning_rate=1)

In [ ]:
### The tf.function 
@tf.function
def train_on_batch(X, y):
    
    with tf.GradientTape() as tape:
        
        y_pred_p_a, y_pred_velocity, sampled_p_a = lstm(X, training=True)
        
        loss_p_a        = losses[0](y, y_pred_p_a)
        loss_velocity   = losses[1](y, y_pred_velocity)
        loss_total      = loss_p_a + tf.math.sqrt(loss_velocity) / 127 #scale the loss to same scale???
        
        metric_p_a      = metrics[0](y, y_pred_p_a)
        confs           = metrics[1](tf.reshape(y[:,:,:,0:2],  [-1,2]), 
                                    tf.reshape(sampled_p_a, [-1,2]))
        p_conf, a_conf  = confs[0], confs[1]
        metric_velocity = metrics[2](y, y_pred_velocity)
        
    grads = tape.gradient(loss_total, lstm.trainable_weights)
    optimizer.apply_gradients(zip(grads, lstm.trainable_weights))
    
    return (tf.reduce_mean(loss_p_a), 
            tf.reduce_mean(tf.math.sqrt(loss_velocity) / 127), 
            tf.reduce_mean(metric_p_a), 
            tf.reduce_mean(metric_velocity), 
            [p_conf, a_conf])

@tf.function
def validate_on_batch(X, y):
    
    y_pred_p_a, y_pred_velocity, sampled_p_a = lstm(X, training=False)
    
    loss_p_a       = losses[0](y, y_pred_p_a)
    loss_velocity  = losses[1](y, y_pred_velocity)
    loss_total     = loss_p_a + tf.math.sqrt(loss_velocity) / 127
    
    metric_p_a     = metrics[0](y, y_pred_p_a)
    confs          = metrics[1](tf.reshape(y[:,:,:,0:2],  [-1,2]), 
                                tf.reshape(sampled_p_a, [-1,2]))
    p_conf, a_conf = confs[0], confs[1]    
    metric_velocity = metrics[2](y, y_pred_velocity)
    
    return (tf.reduce_mean(loss_p_a), 
            tf.reduce_mean(tf.math.sqrt(loss_velocity) / 127), 
            tf.reduce_mean(metric_p_a), 
            tf.reduce_mean(metric_velocity), 
            [p_conf, a_conf])

In [ ]:
n_train_batches = 2048
n_val_batches = 256

epochs = 16
batch_size = 2
epoch_save_list = [1, 2, 4, 8 ,16, 32, 64, 128, 256, 512]

# Values for loss, metric and confusion matrix
train_loss_p_a_array   = np.full((epochs, n_train_batches), 10.0)
train_loss_vel_array   = np.full((epochs, n_train_batches), 10.0)
train_metric_p_a_array = np.full((epochs, n_train_batches), 10.0)
train_metric_vel_array = np.full((epochs, n_train_batches), 10.0)
val_loss_p_a_array     = np.full((epochs, n_val_batches), 10.0)
val_loss_vel_array     = np.full((epochs, n_val_batches), 10.0)
val_metric_p_a_array   = np.full((epochs, n_val_batches), 10.0)
val_metric_vel_array   = np.full((epochs, n_val_batches), 10.0)
train_p_conf_array     = np.full((epochs, 2, 2), 10.0)
train_a_conf_array     = np.full((epochs, 2, 2), 10.0)
val_p_conf_array       = np.full((epochs, 2, 2), 10.0)
val_a_conf_array       = np.full((epochs, 2, 2), 10.0)

In [ ]:
### CUSTOM TRAINING LOOP

# Timing
start_time = time.time()
time_old = start_time

for epoch in range(epochs):
    print('\rStart of Epoch [%d/%d]'% (epoch + 1, epochs))
    print('\n')

    #ensure that every epoch the same training data is used
    random.seed(1337)
    for n in range(n_train_batches):
        print('Training batch: %d/%d' % (n + 1, n_train_batches), end='\r')
        train_dataset = prep.createDataSet(training_pieces, batch_size, num_timesteps, batch_size)
        for _, (X_train, y_train) in enumerate(train_dataset):
            l_1, l_2, m_1, m_2, confusion_mat = train_on_batch(X_train, y_train)
            train_loss_p_a_array[epoch, n]    = l_1
            train_loss_vel_array[epoch, n]    = l_2
            train_metric_p_a_array[epoch, n]  = m_1
            train_metric_vel_array[epoch, n]  = m_2
    print('')
    
    # storing the confusion matrix of validation set for predicting play/ articulate
    train_p_conf_array[epoch, : , :] = confusion_mat[0]
    train_a_conf_array[epoch, : , :] = confusion_mat[1]

    # reset metrics
    metrics[0].reset_states()
    metrics[1].reset_states()

    for n in range(n_val_batches):
        print('Validation batch: %d/%d' % (n + 1, n_val_batches), end='\r')
        val_dataset   = prep.createDataSet(validation_pieces, batch_size, num_timesteps, batch_size)
        for _, (X_val, y_val) in enumerate(val_dataset):
            l_1, l_2, m_1, m_2, confusion_mat = validate_on_batch(X_val, y_val)
            val_loss_p_a_array[epoch, n]      = l_1
            val_loss_vel_array[epoch, n]      = l_2
            val_metric_p_a_array[epoch, n]    = m_1 
            val_metric_vel_array[epoch, n]    = m_2 

    print('')
    print('Seed:' + str(random.randrange(0,1000000,1)))

    # storing the confusion matrix of validation set for predicting play/ articulate
    val_p_conf_array[epoch, : , :] = confusion_mat[0]
    val_a_conf_array[epoch, : , :] = confusion_mat[1]

    # reset metrics
    metrics[0].reset_states()
    metrics[1].reset_states()
    
    print('Training   Loss p_a: '     + str(np.mean(train_loss_p_a_array[epoch,:])))
    print('Validation Loss p_a: '     + str(np.mean(val_loss_p_a_array[epoch,:])))
    print('Training   Loss vel: '     + str(np.mean(train_loss_vel_array[epoch,:])))
    print('Validation Loss vel: '     + str(np.mean(val_loss_vel_array[epoch,:])))
    print('Training   Accuracy p_a: ' + str(np.mean(train_metric_p_a_array[epoch,:])))
    print('Validation Accuracy p_a: ' + str(np.mean(val_metric_p_a_array[epoch,:])))
    print('Training   Accuracy vel: ' + str(np.mean(train_metric_vel_array[epoch,:])))
    print('Validation Accuracy vel: ' + str(np.mean(val_metric_vel_array[epoch,:])))
    print('Training Confusion Matrix Play:\n')
    print(train_p_conf_array[epoch, : , :])
    print('')
    print('Training Confusion Matrix Articulate:\n')
    print(train_a_conf_array[epoch, : , :])
    print('')
    print('Validation Confusion Matrix Play:\n')
    print(val_p_conf_array[epoch, : , :])
    print('')
    print('Validation Confusion Matrix Articulate:\n')
    print(val_a_conf_array[epoch, : , :])
    print('')


    time_new = time.time()
    duration = time_new - time_old
    time_old = time_new
    print('Time: ' +  str(round(duration, 3)) + 's')


    if (epoch + 1) in epoch_save_list:

        # save model weights
        iteration_name = 'epoch_' + str(epoch + 1) + '_dense_dropout'
        save_path = Checkpoint_Directory + current_time_str[:-7] + '/' +  iteration_name + 'model'
        lstm.save(save_path)

        # save audios for train from y_pred_train, y_train
        y_pred_note_train, y_pred_velocity_train, _ = lstm(X_train, training=False)
        y_pred_train = tf.concat([y_pred_note_train, y_pred_velocity_train], axis=-1)
        midi.generate_audio(y_pred_train, 
                            Music_Out_Training_Directory + current_time_str[:-7] + '/', 
                            iteration_name + 'train_pred', 
                            sample=True)
        midi.generate_audio(y_train, 
                            Music_Out_Training_Directory + current_time_str[:-7] + '/', 
                            iteration_name + 'train_true', 
                            sample=False)
        
        # save audios for val from y_pred
        y_pred_note_val, y_pred_velocity_val, _ = lstm(X_val, training=False)
        y_pred_val = tf.concat([y_pred_note_val, y_pred_velocity_val], axis=-1)
        midi.generate_audio(y_pred_val,    
                            Music_Out_Training_Directory + current_time_str[:-7] + '/', 
                            iteration_name + 'val_pred', 
                            sample=True)
        midi.generate_audio(y_val,    
                            Music_Out_Training_Directory + current_time_str[:-7] + '/', 
                            iteration_name + 'val_true', 
                            sample=False)

        # save the arrays 
        save_path = Numpy_Directory + current_time_str[:-7] + '/' 
        try:
            os.mkdir(save_path)    
        except:
            print('destination folder exists')
        np.savez(save_path + iteration_name + 'array', 
                 train_loss_p_a_array, 
                 train_loss_vel_array,
                 train_metric_p_a_array,
                 train_metric_vel_array,
                 val_loss_p_a_array, 
                 val_loss_vel_array,
                 val_metric_p_a_array,
                 val_metric_vel_array,
                 train_p_conf_array,
                 train_a_conf_array,
                 val_p_conf_array,
                 val_a_conf_array,
                 X_train.numpy(),
                 y_train.numpy(),
                 y_pred_train.numpy(),
                 X_val.numpy(),
                 y_val.numpy(),
                 y_pred_val.numpy()
                )
    print('\n\n')

total_time = time.time() - start_time 
print('Total time: ' + str(datetime.timedelta(seconds=total_time)))

### Load model

In [ ]:
load_path = Checkpoint_Directory + '/20210418/epoch_4_dense_dropoutmodel'
model = tf.keras.models.load_model(load_path)

#### Compare precitions to labels

In [ ]:
print_labels = False # True
if print_labels:
    y_pred_note, y_pred_velocity, _ = model.predict(X)
    y_pred = tf.concat([y_pred_note, y_pred_velocity], axis=-1)
    tmp = tfp.distributions.Bernoulli(logits=y_pred[batch_idx,:,time_idx,0:2]).sample()
    tmp2 = tf.cast(np_arrays['y_pred_train'][batch_idx,:, time_idx,2:3], dtype=tf.int32)
    tmp3 = tf.concat([tmp,tmp2], axis=-1)
    print(tf.concat([tf.cast(y[batch_idx,:, time_idx,:], dtype=tf.int32), tmp3], -1))

### MIDI generation

In [ ]:
# Generate additional audios for each batch test audios for each batch
add_for_each_batch = False  # True
if add_for_each_batch:
    n_train_batches = 10
    n_val_batches = 2

    random.seed(1337)
    for n in range(n_train_batches):
        print('Generate training audio: %d/%d' % (n + 1, n_train_batches), end='\r')
        train_dataset = prep.createDataSet(training_pieces, batch_size, num_timesteps, batch_size)
        for _, (X, y) in enumerate(train_dataset):
            y_pred_note, y_pred_velocity, _ = model.predict(X)
            y_pred = tf.concat([y_pred_note, y_pred_velocity], axis=-1)
            midi.generate_audio(y_pred,    Music_Out_Directory, current_time_str, 'batch_' + str(n) + '_train_pred', sample=True, tickscale=50)
            midi.generate_audio(y,    Music_Out_Directory, current_time_str, 'batch_' + str(n) + '_train_true', sample=False, tickscale=50)

    for n in range(n_val_batches):
        print('Generate validation audio: %d/%d' % (n + 1, n_val_batches), end='\r')
        val_dataset   = prep.createDataSet(validation_pieces, batch_size, num_timesteps, batch_size)
        for _, (X, y) in enumerate(val_dataset):
            y_pred_note, y_pred_velocity, _ = model.predict(X)
            y_pred = tf.concat([y_pred_note, y_pred_velocity], axis=-1)
            midi.generate_audio(y_pred,    Music_Out_Directory, current_time_str, 'batch_' + str(n) + '_val_pred', sample=True, tickscale=50)
            midi.generate_audio(y,    Music_Out_Directory, current_time_str, 'batch_' + str(n) + '_val_true' , sample=False, tickscale=50)


#### Genereate new MIDI files from scratch using the trained model

In [ ]:
# Music Generation from scratch

num_notes = 88
num_timesteps = 144
n_beats = 48
n_bars = 10
t_gen = n_bars*n_beats
batch_size_gen = 4
num_timesteps_initial = 1 # start with initial Note_State_Batch with 't' dimension = 1 (can still a batch of samples run in parallel)




notes_gen_initial = tf.zeros((batch_size_gen, num_notes, num_timesteps_initial, 3))

# Initial States
note_state_matrix_gen = notes_gen_initial


# Generate note_state_matrix
for t in tf.range(t_gen):
    if(t<num_timesteps):
        time_init = 0
    else:
        time_init = t%48
    X  = prep.inputKernel(note_state_matrix_gen[:,:,-num_timesteps:,:], time_init=time_init)
    _ , y_pred_velocity_train, y_pred_note_train = model.predict_on_batch(X)
    new_note = tf.concat([y_pred_note_train[:,:,-1:,:], y_pred_velocity_train[:,:,-1:,:]], axis=-1)
    new_note_p   = new_note[:,:,:,0]
    new_note_a   = new_note[:,:,:,1] * new_note[:,:,:,0]
    new_note_vel = new_note[:,:,:,2] * new_note[:,:,:,0]
    new_note = tf.stack([new_note_p, new_note_a, new_note_vel], axis=-1)
    note_state_matrix_gen = tf.concat([note_state_matrix_gen, new_note], axis=2)

In [ ]:
for i in range(batch_size_gen):
    midi.generate_audio(note_state_matrix_gen[i:(i+1),:,:,:], 
                        Music_Out_Genereating_Directory + current_time_str[:-7] + '/',
                        'generated' + '_batch_' + str(i) + '_epoch_4', 
                        sample=False,
                        verbose = False)

In [ ]:
# look at feature vector
prep.noteRNNInputSummary(prep.inputKernel(note_state_matrix_gen[:,:,0:144,:], time_init=0)[1,56,2,:])

### Analyze Results

In [ ]:
filename = 'epoch_16_dense_dropoutarray.npz'
npzfile = np.load(Numpy_Directory + '20210418' + '/' + filename) #  + current_time_str[:-7]
keys = ['train_loss_p_a', 'train_loss_vel','train_metric_p_a', 'train_metric_vel', 
        'val_loss_p_a', 'val_loss_vel', 'val_metric_p_a', 'val_metric_vel', 
        'train_p_conf_array', 'train_a_conf_array','val_p_conf_array', 'val_a_conf_array',
        'X_train', 'y_train', 'y_pred_train', 'X_val', 'y_val', 'y_pred_val'
       ]
np_arrays = {}
for i in range(len(keys)):
    np_arrays[keys[i]] = npzfile['arr_'+ str(i)] 

#### Compare labels to predictions

In [ ]:
note_idx = 30
time_idx = 60
batch_idx = 0

print_labels = False # True
if print_labels:
    tmp = tfp.distributions.Bernoulli(logits=np_arrays['y_pred_train'][batch_idx,:,time_idx,0:2]).sample()
    tmp2 = tf.cast(np_arrays['y_pred_train'][batch_idx,:, time_idx,2:3], dtype=tf.int32)
    tmp3 = tf.concat([tmp,tmp2], axis=-1)
    tf.concat([np_arrays['y_train'][batch_idx,:, time_idx,:], tmp3], -1)

#### Visualisations - Plotly

In [ ]:
max_trained = 16
train_loss_p_a_avg = np.mean(np_arrays['train_loss_p_a'], axis=1)[0:max_trained]
val_loss_p_a_avg = np.mean(np_arrays['val_loss_p_a'], axis=1)[0:max_trained]
fig = px.line(y=[train_loss_p_a_avg,val_loss_p_a_avg])
fig.data[0].name = "train_loss_avg_p_a"
fig.data[1].name = "val_loss_avg_p_a"

fig.show()

In [ ]:
train_metric_avg = np.mean(np_arrays['train_metric_p_a'], axis=1)[0:max_trained]
val_metric_avg = np.mean(np_arrays['val_metric_p_a'], axis=1)[0:max_trained]
fig = px.line(y=[train_metric_avg, val_metric_avg])
fig.data[0].name = "train_metric_avg_p_a"
fig.data[1].name = "val_metric_avg_p_a"

fig.show()

In [ ]:
train_loss_vel_avg = np.mean(np_arrays['train_loss_vel'], axis=1)[0:max_trained]
val_loss_vel_avg = np.mean(np_arrays['val_loss_vel'], axis=1)[0:max_trained]
fig = px.line(y=[train_loss_vel_avg,val_loss_vel_avg])
fig.data[0].name = "train_loss_avg_vel"
fig.data[1].name = "val_loss_avg_vel"
fig.show()

In [ ]:
train_metric_avg = np.mean(np_arrays['train_metric_vel'], axis=1)[0:max_trained]
val_metric_avg = np.mean(np_arrays['val_metric_vel'], axis=1)[0:max_trained]
fig = px.line(y=[train_metric_avg, val_metric_avg])
fig.data[0].name = "train_metric_avg_vel"
fig.data[1].name = "val_metric_avg_vel"

fig.show()

In [ ]:
p_conf_percent = np_arrays['train_p_conf_array'][0:max_trained] / np.sum(np_arrays['train_p_conf_array'][0:max_trained], axis=1, keepdims=True)

fig = px.line(y=[p_conf_percent[:,0,0],p_conf_percent[:,1,0], p_conf_percent[:,0,1], p_conf_percent[:,1,1]])
fig.data[0].name = "pred_not_played_true_not_played"
fig.data[1].name = "pred_played_true_not_played"
fig.data[2].name = "pred_not_played_true_played"
fig.data[3].name = "pred_played_true_played"


fig.show()

In [ ]:
p_conf_percent = np_arrays['val_p_conf_array'][0:max_trained]  / np.sum(np_arrays['val_p_conf_array'][0:max_trained] , axis=1, keepdims=True)

fig = px.line(y=[p_conf_percent[:,0,0],p_conf_percent[:,1,0], p_conf_percent[:,0,1], p_conf_percent[:,1,1]])
fig.data[0].name = "pred_not_played_true_not_played"
fig.data[1].name = "pred_played_true_not_played"
fig.data[2].name = "pred_not_played_true_played"
fig.data[3].name = "pred_played_true_played"


fig.show()

In [ ]:
a_conf_percent = np_arrays['train_a_conf_array'][0:max_trained]  / np.sum(np_arrays['train_a_conf_array'][0:max_trained] , axis=1, keepdims=True)

fig = px.line(y=[a_conf_percent[:,0,0],a_conf_percent[:,1,0], a_conf_percent[:,0,1], a_conf_percent[:,1,1]])
fig.data[0].name = "pred_not_articulated_true_not_articulated"
fig.data[1].name = "pred_articulated_true_not_articulated"
fig.data[2].name = "pred_not_articulated_true_articulated"
fig.data[3].name = "pred_articulated_true_articulated"


fig.show()

In [ ]:
a_conf_percent = np_arrays['val_a_conf_array'][0:max_trained]  / np.sum(np_arrays['val_a_conf_array'][0:max_trained] , axis=1, keepdims=True)

fig = px.line(y=[a_conf_percent[:,0,0],a_conf_percent[:,1,0], a_conf_percent[:,0,1], a_conf_percent[:,1,1]])
fig.data[0].name = "pred_not_articulated_true_not_articulated"
fig.data[1].name = "pred_articulated_true_not_articulated"
fig.data[2].name = "pred_not_articulated_true_articulated"
fig.data[3].name = "pred_articulated_true_articulated"


fig.show()